In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.stores import InMemoryByteStore
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough , RunnableParallel

In [35]:
# Carregando variaveis de ambiente

from dotenv import load_dotenv
load_dotenv()

True

In [24]:
# Define p array de perguntas
perguntas = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"
]

In [25]:
#Carrega o PDF

pdf_link = "../os-sertoes.pdf"

loader =  PyPDFLoader(pdf_link,extract_images=False)

pages = loader.load_and_split()


In [26]:
# carrega o Embedding e modelo llm

embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    max_tokens = 500
)

In [27]:
#Splitters

child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [28]:
#Storages
stores= InMemoryByteStore()
vectorstore = Chroma(embedding_function=embeddings,persist_directory="childVectorDB")


In [29]:
#Document Retriver

parent_document_retriver  = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=stores,
    child_splitter= child_splitter,
    parent_splitter= parent_splitter
)

parent_document_retriver.add_documents(pages, ids=None)


In [30]:
TEMPLATE = """
    Você é um experiente analisador de obras literárias. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""

In [31]:
# Prompt

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [32]:
#Retrivel

setup_retrivel = RunnableParallel({"question":RunnablePassthrough(), "context": parent_document_retriver})
output_parser = StrOutputParser()

In [33]:
#Chain
parent_chain_retrivel = setup_retrivel | rag_prompt | llm | output_parser

In [34]:
for p in perguntas:
    resposta = parent_chain_retrivel.invoke(p)
    print(f"Pergunta:\n{p}\n")
    print(f"Resposta:\n{resposta}")
    print("-" * 80)

Pergunta:
Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?

Resposta:
Euclides da Cunha apresenta em sua obra uma visão mesológica do ambiente natural do sertão nordestino, destacando a influência dos elementos físicos, como o clima, na vida dos habitantes da região. Ele ressalta a brutalidade do ambiente, a escassez de recursos naturais e as condições adversas que moldam a vida dos sertanejos. Além disso, o autor aponta a indiferença e a inércia da população em relação às questões ambientais, o que contribui para a perpetuação das dificuldades enfrentadas no sertão. Euclides da Cunha aborda como o clima, a geologia e a topografia contribuem para as condições de vida no sertão e como esses fatores se interrelacionam, criando um ciclo de influências recíprocas que impactam diretamente a existência dos habitantes da região.
--------------------------------------------------------------------------------
Pergunt